# Automatic Quiz Generator

In [24]:
import os
import openai

### Set-up OpenAI API key from environment variables

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

### Familiarize GPT with the format of the test

In [4]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions. " \
                 + f"Each question should have {num_possible_answers} options. "\
                 + f"Also include the correct answer for each question using the starting string 'Correct Answer: '."
    return prompt

In [5]:
create_test_prompt("Python", 4, 4)

"Create a multiple choice quiz on the topic of Python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer: '."

### OpenAI API Call

In [7]:
response = openai.Completion.create(engine="text-davinci-003",
                                            prompt=create_test_prompt("Python", 4, 4),
                                            max_tokens=256,
                                            temperature=0.7)

In [8]:
response

<OpenAIObject text_completion id=cmpl-6w0DUDXem4vw22BGRZwJXiKdta0HH at 0x273801b1900> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nQ1. What type of language is Python?\nA. Compiled\nB. Scripted\nC. Object-oriented\nD. Machine language\nCorrect Answer: B. Scripted\n\nQ2. What is the primary use of Python?\nA. Web development\nB. Mobile application development\nC. Desktop application development\nD. Data analysis\nCorrect Answer: A. Web development\n\nQ3. Which of the following is not a core data type in Python?\nA. Lists\nB. Strings\nC. Tuples\nD. Classes\nCorrect Answer: D. Classes\n\nQ4. What is the correct syntax to print the word 'Hello' in Python?\nA. println(Hello)\nB. print('Hello')\nC. printf(Hello)\nD. log('Hello')\nCorrect Answer: B. print('Hello')"
    }
  ],
  "created": 1679281876,
  "id": "cmpl-6w0DUDXem4vw22BGRZwJXiKdta0HH",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": 

### Parse the required information (QnA) from response

In [9]:
response["choices"][0]["text"]

"\n\nQ1. What type of language is Python?\nA. Compiled\nB. Scripted\nC. Object-oriented\nD. Machine language\nCorrect Answer: B. Scripted\n\nQ2. What is the primary use of Python?\nA. Web development\nB. Mobile application development\nC. Desktop application development\nD. Data analysis\nCorrect Answer: A. Web development\n\nQ3. Which of the following is not a core data type in Python?\nA. Lists\nB. Strings\nC. Tuples\nD. Classes\nCorrect Answer: D. Classes\n\nQ4. What is the correct syntax to print the word 'Hello' in Python?\nA. println(Hello)\nB. print('Hello')\nC. printf(Hello)\nD. log('Hello')\nCorrect Answer: B. print('Hello')"

## Q/A Extraction

### Extract Student View (Only Questions)

In [10]:
def create_student_view(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view

In [11]:
create_student_view(response["choices"][0]["text"], 4)

{1: '\n\nQ1. What type of language is Python?\nA. Compiled\nB. Scripted\nC. Object-oriented\nD. Machine language\n',
 2: '\nQ2. What is the primary use of Python?\nA. Web development\nB. Mobile application development\nC. Desktop application development\nD. Data analysis\n',
 3: '\nQ3. Which of the following is not a core data type in Python?\nA. Lists\nB. Strings\nC. Tuples\nD. Classes\n',
 4: "\nQ4. What is the correct syntax to print the word 'Hello' in Python?\nA. println(Hello)\nB. print('Hello')\nC. printf(Hello)\nD. log('Hello')\n"}

### Extract Answers Only

In [12]:
def extract_answers(test, num_questions):
    answers = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+"\n"

            if question_number < num_questions:
                question_number+=1
                answers[question_number] = ""
    return answers

In [13]:
extract_answers(response["choices"][0]["text"], 4)

{1: 'Correct Answer: B. Scripted\n',
 2: 'Correct Answer: A. Web development\n',
 3: 'Correct Answer: D. Classes\n',
 4: "Correct Answer: B. print('Hello')\n"}

### Exam Simulation

In [14]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers

In [21]:
student_answers = take(create_student_view(response["choices"][0]["text"], 4))



Q1. What type of language is Python?
A. Compiled
B. Scripted
C. Object-oriented
D. Machine language

Enter your answer: b

Q2. What is the primary use of Python?
A. Web development
B. Mobile application development
C. Desktop application development
D. Data analysis

Enter your answer: d

Q3. Which of the following is not a core data type in Python?
A. Lists
B. Strings
C. Tuples
D. Classes

Enter your answer: d

Q4. What is the correct syntax to print the word 'Hello' in Python?
A. println(Hello)
B. print('Hello')
C. printf(Hello)
D. log('Hello')

Enter your answer: b


### Automatic Grading

In [22]:
def grade(correct_answer_dict, answers):
    correct_answers = 0
    for question, answer in answers.items():
        if answer.upper() == correct_answer_dict[question].upper()[16]:
            correct_answers+=1
    grade = 100 * correct_answers / len(answers)

    if grade < 60:
        passed = "Not passed!"
    else:
        passed = "Passed!"
    return f"{correct_answers} out of {len(answers)} correct! You achieved: {grade} % : {passed}"

In [23]:
grade(extract_answers(response["choices"][0]["text"], 4), student_answers)

'3 out of 4 correct! You achieved: 75.0 % : Passed!'